In [1]:
import os
import joblib
import pandas as pd
import re
import sys
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
def preprocess_text(text):
    # Suppression des caractères spéciaux
    text = re.sub(r"[^a-zA-ZÀ-ÿ0-9\s]", "", text)
    
    # Suppression des espaces après la suppression des caractères spéciaux
    text = text.strip()
    
    # Suppression de la ponctuation
    text = re.sub(r"[^\w\s]", "", text)
    # Suppression des chiffres
    text = re.sub(r"\d", "", text)
    text = text.replace('&', '')
    # Convertir le texte en minuscules
    text = text.lower()
    # Remplacer les doubles espaces par un seul espace
    text = re.sub(r'\s+', ' ', text)
  
  
    return text

In [3]:
def charger_fichiers_csv(chemin):
    # Obtenir la liste des fichiers dans le chemin spécifié
    fichiers = os.listdir(chemin)
    
    dataframes = []
    
    # Parcourir chaque fichier dans la liste
    for fichier in fichiers:
        if fichier.endswith('.csv'):  # Vérifier si le fichier a l'extension .csv
            chemin_fichier = os.path.join(chemin, fichier)  # Chemin complet du fichier
            
            # Charger le fichier CSV dans un DataFrame et l'ajouter à la liste
            df = pd.read_csv(chemin_fichier)
            dataframes.append(df)

     # Combinaison des DataFrames en un seul
    df = pd.concat(dataframes, ignore_index=True)

    return df

In [4]:

def regrouper_categories(df, mots_cles):
    
    # Fonction pour vérifier si un mot-clé est présent dans une catégorie
    def trouver_mot_cle(categorie, mots_cles):
        for mot_cle in mots_cles:
            if mot_cle in categorie.lower():
                return mot_cle
        return None

    # Appliquer la fonction trouver_mot_cle à chaque catégorie
    df['category'] = df['category'].astype(str).apply(lambda x: trouver_mot_cle(x, mots_cles))

    return df

In [5]:
# -Vectorisation des données 
#- la recherche des meilleurs paramètres à l'aide de la validation croisée
#-entraînement du modèle LinearSVC
#-la prédiction des labels sur l'ensemble de test

In [6]:
def train_linear_svc_model(df):
    df['category'] = df['category'].fillna('')
    df['description'] = df['description'].fillna('')
    # Diviser les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement, 20% pour le test)
    X_train, X_test, y_train, y_test = train_test_split(df['description'], df['category'], test_size=0.2, random_state=42)
    
    X_train = X_train.apply(preprocess_text)
    X_test = X_test.apply(preprocess_text)
    
    # Créer une instance de Tf-idfVectorizer ou CountVectorizer
    tfidf_vectorizer = TfidfVectorizer()
    count_vectorizer = CountVectorizer()
    
    # Appliquer la vectorisation aux données d'entraînement et de test
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    X_train_count = count_vectorizer.fit_transform(X_train)
    X_test_count = count_vectorizer.transform(X_test)
    
    # Définir une pipeline avec une vectorisation Tfidf et un classificateur LinearSVC
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LinearSVC())
    ])
    
    # Définir les paramètres à tester pour le classificateur LinearSVC
    parameters = {
        'clf__C': [0.1, 1, 10],
        'clf__max_iter': [100, 500, 1000]
    }
    
    # Cross validation
    # Utiliser la validation croisée pour trouver les meilleurs paramètres
    grid_search = GridSearchCV(pipeline, parameters, cv=5)
    grid_search.fit(X_train, y_train)
    
    # Afficher les meilleurs paramètres et le score obtenu sur l'ensemble de validation
    print("Meilleurs paramètres : ")
    print(grid_search.best_params_)
    print("Score sur l'ensemble de validation : ")
    print(grid_search.best_score_)
    
    # Créer une instance du classificateur SVM à noyau linéaire avec les meilleurs paramètres trouvés
    best_clf = LinearSVC(C=grid_search.best_params_['clf__C'], max_iter=grid_search.best_params_['clf__max_iter'])
    
    # Entraîner le modèle sur l'ensemble d'entraînement
    best_clf.fit(X_train_tfidf, y_train)
    
    # Prédire les labels sur l'ensemble de test
    predictions_SVM = best_clf.predict(X_test_tfidf)
    
    return best_clf, X_test_tfidf, y_test

In [7]:
def evaluate_and_save_model(best_clf, X_test_tfidf, y_test, model_path):
    # Faire des prédictions sur l'ensemble de test
    y_pred = best_clf.predict(X_test_tfidf)

    # Calculer la précision
    precision = precision_score(y_test, y_pred, average='weighted')
    print("Precision:", precision)

    # Calculer le score F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Score F1 sur l'ensemble de test:", f1)
    sys.stdout.flush()
    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy sur l'ensemble de test:", accuracy)

    # Enregistrer le modèle
    joblib.dump(best_clf, model_path)


In [8]:
def main():
    chemin_dossier = "C:/Users/infoMix/Desktop/PFE/Deutch"
    
    # Charger les fichiers CSV
    df = charger_fichiers_csv(chemin_dossier)
    
    mots_cles = ['pflege', 'makeup', 'sonne', 'körper', 'düfte', 'haare', 'gesicht', 'gesundheit',
                 'lebensmittel', 'tier', 'baby spielzeug', 'home lifestyle', 'parfum',
                 'apotheke gesundheit', 'haushalt', 'ernährung']
    
    # Prétraitement des colonnes 'category' et 'description'
    df['category'] = df['category'].astype(str)
    df['description'] = df['description'].astype(str)
    df['category'] = df['category'].apply(preprocess_text)
    df['description'] = df['description'].apply(preprocess_text)
    df = df.dropna()
    
    # Regrouper les catégories en utilisant des mots-clés
    df = regrouper_categories(df, mots_cles)
    
    # Entraîner le modèle en utilisant la fonction train_linear_svc_model
    best_clf, X_test_tfidf, y_test = train_linear_svc_model(df)
    
    # Évaluer le modèle et enregistrer le modèle entraîné
    evaluate_and_save_model(best_clf, X_test_tfidf, y_test, 'model.pkl')

# Appel de la fonction main
if __name__ == "__main__":
    main()

C:\Users\infoMix\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\infoMix\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\infoMix\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\infoMix\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\infoMix\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\infoMix\A

Meilleurs paramètres : 
{'clf__C': 10, 'clf__max_iter': 500}
Score sur l'ensemble de validation : 
0.9368635437881874
Precision: 0.9404534504680379
Score F1 sur l'ensemble de test: 0.9404190751981092
Accuracy sur l'ensemble de test: 0.9410806407819712


C:\Users\infoMix\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\infoMix\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
